
Hi! I am solving Project Euler problem 479.

The problem is as follows.

> For some $k$, define $a_k, b_k, c_k$ as the three solutions to the equation $$\frac{1}{x} = \left(\frac{k}{x}\right)^2 (k + x^2) - kx$$
> Let $$S(n) = \sum_{p=1}^n \sum_{k=1}^n (a_k + b_k)^p(b_k + c_k)^p(c_k + a_k)^p$$ 
$S(n)$ is always an integer. $S(4) = 51160$. Evaluate $S(10^6) \mod (10^9 + 7)$.

I start by expanding the equation first.

$$
\frac{1}{x} = \left(\frac{k}{x}\right)^2 (k + x^2) - kx = \frac{k^2}{x^2}(k + x^2) - kx \\~\\= \frac{k^3}{x^2} + k^2 - kx \\~\\
\Rightarrow 1 = \frac{k^3}{x} + k^2x - kx^2 \Rightarrow x = k^3 + k^2x^2 - kx^3 \\~\\
\Rightarrow kx^3 -k^2x^2 + x - k^3 = 0
$$

Let $a_k = a, b_k = b, c_k = c$. Using [Viete's formula](https://en.wikipedia.org/wiki/Vieta%27s_formulas#Example):

$$
\begin{cases}
a + b + c = k \\
ab + bc + ac = \frac{1}{k} \\
abc = k^2 
\end{cases}
$$

---

Now, consider the product:

$$
(a + b)(b + c)(a + c) = (a + b)(ab + ac +bc +c^2) \\~\\ = a^2b + a^2c + abc + ac^2 + ab^2 + abc + b^2c + bc^2 \\~\\ = (a^2b + ab^2 + abc  + b^2c + bc^2 + abc + a^2c + ac^2 + abc) - abc \\~\\ = ab(a + b + c) + bc(a + b + c) + ac(a + b + c) - abc \\~\\ = (ab + bc + ac)(a + b + c) - abc \\~\\ = \frac{1}{k} \times k - k^2  = 1 - k^2
$$

and 

$$
S(n) = \sum_{p=1}^n \sum_{k=1}^n (a_k + b_k)^p(b_k + c_k)^p(c_k + a_k)^p \\~\\ = \sum_{p=1}^n \sum_{k=1}^n \left[(a_k + b_k)(b_k + c_k)(c_k + a_k)\right]^p = \sum_{p=1}^n \sum_{k=1}^n \left(1-k^2\right)^p = \sum_{k=1}^n \sum_{p=1}^n \left(1-k^2\right)^p \\~\\ = \sum_{k=1}^n \left(\sum_{p=1}^n \left(1-k^2\right)^p \right)
$$

Consider the geometric sum $\displaystyle{\sum_{p=1}^n \left(1-k^2\right)^p}$. Define $a = 1 - k^2$ and $$\displaystyle{\sum_{p=1}^n \left(1-k^2\right)^p = \sum_{p=1}^n a^p = \frac{a^{n + 1} - 1}{a - 1} - 1}$$

Therefore $$S(n) = \sum_{k=1}^n \left[\frac{(1-k^2)^{n + 1} - 1}{(1 - k^2) - 1} - 1\right]$$

In [8]:
def fastpow(a: int, b: int) -> int:
    t = 1
    while b > 0:
        if b % 2 == 1:
            t = t * a
        a *= a
        b //= 2
    return t

def S(n: int) -> int:
    t = 0
    for k in range(1, n + 1):
        t += (fastpow(1 - k ** 2, n + 1) - 1) // (-k * k) - 1
    return t

S(10 ** 5)

KeyboardInterrupt: 

But the problem is that we need to implement this modulo $10^9 + 7$. Well, first we trim off some parts: 
$$S(n) = \sum_{k=1}^n \left[\frac{(1-k^2)^{n + 1} - 1}{(1 - k^2) - 1} - 1\right] \\~\\ = \sum_{k=1}^n \left[\frac{(1-k^2)^{n + 1} - 1}{-k^2} - 1\right]$$ 

Now we need to worry about signs, but that's ez enough

In [9]:
def fastpow_with_sign(a: int, b: int) -> tuple[int, int]:
    sign = 0
    if a > 0: sign = 1
    elif a < 0: sign = 1 if b % 2 == 0 else -1
    
    return (fastpow(abs(a), abs(b)), sign)

Also we take modulo $10^9 + 7$ so we can simply use modular inverses in place of division.

Small little proof: Let there be integers $a, b, n$ such that $b = na$ and a prime $p$ that divides none of $a, n, b$ then 

$$
\frac{an}{a} \equiv an \cdot a^{-1} \equiv n \pmod p  
$$

In [7]:
pow(-2, -1, 5)

2

In [10]:
# wait
# so you're telling me Python's pow function support negative inputs
n = 10 ** 6
mod = 10 ** 9 + 7
t = 0

for k in range(1, n + 1):
    z = (pow(1 - k * k, n + 1, mod) - 1) % mod
    z *= pow(- k * k, -1, mod)
    z -= 1
    t += z % mod
t % mod

191541795